In [1]:
import pandas as pd
import numpy as np 
import matplotlib as plt 

tweets = pd.read_excel('AstonVillaTweetsMOS_NP.xlsx', engine='openpyxl')
tweets.head(5)

,Source.Name,Column1,id,date,source,place,username,Content,hashtags,Likes
0,avfcMOS_analysisV1hashtag.xlsx,0,1467277611395129088,2021-12-04 23:40:17,"<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,AVFCFaithful_,https://t.co/NsfOukd8LY\n\nWe look ahead to to...,['avfc'],16
1,avfcMOS_analysisV1hashtag.xlsx,1,1467272908577852928,2021-12-04 23:21:36,"<a href=""http://twitter.com/download/iphone"" r...",NaN,VillaFiend,"Apart from Ashley Young, how many other free t...","['AVFC', 'AstonVilla', 'UTV']",1
2,avfcMOS_analysisV1hashtag.xlsx,2,1467270944699858944,2021-12-04 23:13:48,"<a href=""http://twitter.com/download/iphone"" r...",NaN,travillafc,#mancity fans singing about gerrard against #w...,"['mancity', 'watford', 'avfc']",179
3,avfcMOS_analysisV1hashtag.xlsx,3,1467268508518134016,2021-12-04 23:04:07,"<a href=""http://twitter.com/download/iphone"" r...",NaN,academyavfc,There’s a starman running down the right 🤩 @Ka...,['AVFC'],425
4,avfcMOS_analysisV1hashtag.xlsx,4,1467267415025274880,2021-12-04 22:59:46,"<a href=""http://twitter.com/download/android"" ...",NaN,TheVillaYamYam,G'wan Tayls #avfc #UTV,"['avfc', 'UTV']",8


In [2]:
import re
from nltk.corpus import stopwords

def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    # Remove mentions
    text = re.sub(r'@\w+', '', text)
    # Remove hashtags
    text = re.sub(r'#\w+', '', text)
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Remove whitespace
    text = text.strip()
    
    return text

In [3]:
import pandas as pd
from sklearn.utils import shuffle

# Apply the preprocessing function to the 'text' column
tweets['processed_text'] = tweets['Content'].apply(preprocess_text)
tweets = shuffle(tweets).reset_index(drop=True)

In [ ]:
tweets.to_csv('AV.csv', index=False)

In [26]:
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Dropout, Bidirectional
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

X = tweets['processed_text']

max_features = 30000
max_length = 100

tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X)
X = pad_sequences(X, maxlen=max_length)

In [27]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, TensorDataset
from torch.autograd import Variable
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
from keras.preprocessing.text import Tokenizer

# Load GloVe embeddings
def load_glove_embeddings(file_path, embedding_dim, word_index, max_features):
    embeddings_index = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    
    embedding_matrix = np.zeros((max_features, embedding_dim))
    for word, i in word_index.items():
        if i >= max_features:
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return torch.tensor(embedding_matrix, dtype=torch.float32)

# Load the GloVe embeddings matrix
glove_file_path = 'glove.twitter.27B.200d.txt'  # Update the path to the downloaded GloVe file
embedding_dim = 200
embedding_matrix = load_glove_embeddings(glove_file_path, embedding_dim, tokenizer.word_index, max_features)

class CNNTLSTM(nn.Module):
    def __init__(self, embedding_matrix, embed_dim=200, lstm_out=448, dropout_rate=0.2, num_classes=3, num_filters=224, filter_size=3, pool_size=2):
        super(CNNTLSTM, self).__init__()
        self.embedding = nn.Embedding.from_pretrained(embedding_matrix, freeze=True)
        self.conv1d = nn.Conv1d(embed_dim, num_filters, filter_size)
        self.relu = nn.ReLU()
        self.max_pool1d = nn.MaxPool1d(pool_size)
        self.bi_lstm = nn.LSTM(num_filters, lstm_out // 2, batch_first=True, bidirectional=True, dropout=dropout_rate)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc = nn.Linear(lstm_out, num_classes)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.embedding(x)
        x = x.permute(0, 2, 1)
        x = self.conv1d(x)
        x = self.relu(x)
        x = self.max_pool1d(x)
        x = x.permute(0, 2, 1)
        x, _ = self.bi_lstm(x)
        x = x[:, -1, :]
        x = self.dropout(x)
        x = self.fc(x)
        return self.softmax(x)
    


In [24]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset, TensorDataset
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Function to load a saved model
def load_saved_model(model_path):
    model = CNNTLSTM(embedding_matrix).cuda()
    model.load_state_dict(torch.load(model_path))
    model.eval()
    return model

# Tokenize and pad the preprocessed texts
X = tweets['processed_text']
max_features = 30000
max_length = 100
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X)
tokenized_texts = tokenizer.texts_to_sequences(X)
padded_texts = pad_sequences(tokenized_texts, maxlen=max_length)

# Convert the tokenized and padded texts to PyTorch tensors
texts_torch = torch.tensor(padded_texts, dtype=torch.long)

# Load the saved model
model_path = 'cnntlstm_modelSP2.pth'
loaded_model = load_saved_model(model_path)

# Use the model to predict sentiment labels for the text data
label_map = {"NEGATIVE": 0, "POSITIVE": 1, "NEUTRAL": 2}

with torch.no_grad():
    # Move the model to the GPU
    loaded_model.cuda()
    
    # Create a DataLoader object to load the text tensors in batches
    dataset = TensorDataset(texts_torch)
    dataloader = DataLoader(dataset, batch_size=512)
    
    # Predict the sentiment labels in batches and concatenate the results
    predictions = []
    for batch in dataloader:
        batch = batch[0].cuda()
        output_probs = loaded_model(batch)
        _, batch_predictions = torch.max(output_probs.data, 1)
        batch_predictions = batch_predictions.cpu().numpy()
        batch_predictions = [label_map[pred] for pred in batch_predictions]
        predictions += batch_predictions

# Add the predicted labels to the original DataFrame and save it to a new CSV file
tweets['predicted_sentiment'] = predictions
tweets.to_csv('TEST3.csv', index=False)


RuntimeError: Error(s) in loading state_dict for CNNTLSTM:
	size mismatch for conv1d.weight: copying a param with shape torch.Size([224, 200, 3]) from checkpoint, the shape in current model is torch.Size([256, 200, 7]).
	size mismatch for conv1d.bias: copying a param with shape torch.Size([224]) from checkpoint, the shape in current model is torch.Size([256]).
	size mismatch for bi_lstm.weight_ih_l0: copying a param with shape torch.Size([896, 224]) from checkpoint, the shape in current model is torch.Size([896, 256]).
	size mismatch for bi_lstm.weight_ih_l0_reverse: copying a param with shape torch.Size([896, 224]) from checkpoint, the shape in current model is torch.Size([896, 256]).